# Problem Description

Consider the diophantine equation $\frac{1}{a}+\frac{1}{b}= \frac{p}{10^n}$ with $a, b, p, n$ positive integers and $a ≤ b$.

How many solutions has this equation for $1 ≤ n ≤ 9$?

Rearrange equation as follows:

$10^n(a + b) = pab$

For a given value of $a$ and $p$ we can find the value of $b$ as follows:

$b = \frac{(10^n a)}{(p \cdot a - 10^n)}$

For a given value of $a$ we can find the minimum value of $p$ as follows:

$p_{min} = \frac{10^n}{a}$ (this follows from the fact that $p \cdot a - 10^n > 0$)

The minimum value for $b$ is when $b=a$, from which we can rearrange the above to get a mazimum value of $p$ as follows:

$b = a = \frac{(10^n a)}{(p \cdot a - 10^n)} \Rightarrow p = \frac{2 \cdot 10^n}{a}$

Solutions $a, b, p$ for $n = 1$ are:

$1, 1, 20$

$1, 2, 15$

$1, 5, 12$

$1, 10, 11$

$2, 2, 10$

$2, 5, 7$

$2, 10, 6$

$3, 6, 5$

$3, 15, 4$

$4, 4, 5$

$4, 20, 3$

$5, 5, 4$

$5, 10, 3$

$6, 30, 2$

$10, 10, 2$

$11, 110, 1$

$12, 60, 1$

$14, 35, 1$

$15, 30, 1$

$20, 20, 1$

# Brute Force Approach 

In [ ]:
def solve(n):
    infer = lambda p, a: (10**n * a) / (p * a - 10**n)
    def solve_for_a(a):
        solutions = 0
        p = int(10**n / a) + 1
        b = infer(p, a)
        while b >= a:
            if (abs(int(b) - b) == 0):
                solutions += 1
            p += 1
            b = infer(p, a)
        return solutions
    a = 1
    solutions = 0
    while True:
        solutions += solve_for_a(a)
        a += 1
        p = int(10**n / a) + 1
        b = infer(p, a)
        if b < a:
            break
    return solutions

# Look into prime factorization

In [ ]:
from nmutils.primes import get_prime_factors
def factorize(n):
    factors = get_prime_factors(n)
    if len(factors) > 1:
        return [int(factor) for factor in factors]
    return factors[0] if factors else 1

In [ ]:
solutions = [[1, 1, 20], [1, 2, 15], [1, 5, 12], [1, 10, 11],
             [2, 2, 10], [2, 5, 7], [2, 10, 6], [3, 6, 5],
             [3, 15, 4], [4, 4, 5], [4, 20, 3], [5, 5, 4],
             [5, 10, 3], [6, 30, 2], [10, 10, 2], [11, 110, 1],
             [12, 60, 1], [14, 35, 1], [15, 30, 1], [20, 20, 1]]

In [ ]:
for solution in solutions:
    a, b, p = solution
    print(solution, f"\t-->\t10 x ({a} + {b}) = {p} x {a} x {b}")
    print([factorize(x) for x in solution])
    print('-' * 60)

Fix $a = 3, n = 1$ - then $p_{min} = ceiling(\frac{10^1}{3}) = 4$. If $p = 4$ then $b = \frac{(10^1 \cdot 3)}{(4 \cdot 3 - 10^1)} = \frac{30}{2} = 15$.

So for $a = 3$ and $n = 1$ we have $p \geq 4$ and $b \leq 15$.

Let's look at values of $b$ in the range $[3, 15]$ and see what we get!

In [ ]:
a = 3
for b in range(15, 2, -1):
    print(f"10 x ({a} + {b}) = {10 * (a + b)} = p x {a} x {b} --> p = {10 * (a + b) / (a * b):.3f} --> {10 * (a + b) % (a * b) == 0}")

From this we can see that if $p$ is to divide $10(a+b)$ then the product $a \cdot b$ must divide $10(a+b)$.

In other words, for given $n$ and $a$, we have a solution for $b$ precisely when $10^n(a + b) \mod a \cdot b = 0$!

Does this pose a solution?

For given $n$:
<ol>
    <li>Start with $a=1$</li>
    <li>Calculate $p_{min} = ceiling(\frac{10^n}{a})$</li>
    <li>Calculate $b_{max} = \frac{(10^n a)}{(p_{min} \cdot a - 10^n)}$</li>
    <li>$\forall b \in [a, b_{max}]$, count number of solutions where $10^n(a + b) \mod a \cdot b = 0$</li>
    <li>Increment $a$ and repeat</li>
    <li>Stop when $a \gt b$</li>
</ol>

In [ ]:
from itertools import count, takewhile
from math import ceil

In [ ]:
def elegant(n):
    def solve_for_a(a):
        pmin = ceil(10**n / a)
        if pmin * a == 10**n:
            pmin += 1
        bmax = int((10**n * a) / (pmin * a - 10**n))
        if bmax < a:
            return None
        solutions = 0
        for b in range(a, bmax + 1):
            if 10**n * (a + b) % (a * b) == 0:
                solutions += 1
        return solutions
    infinite_solutions = map(solve_for_a, count(1))
    solutions = takewhile(lambda s: s is not None, infinite_solutions)
    return sum(solutions)

In [ ]:
%%time
[elegant(n) for n in range(1, 4)]

# Combine brute force with elegant solution

In [ ]:
def solve2(n):
    infer = lambda p, a: int((10**n * a) / (p * a - 10**n))
    def solve_for_a(a):
        check_solution = lambda b: 10**n * (a + b) % (a * b) == 0
        solutions = set()#0
        p = int(10**n / a) + 1
        b = infer(p, a)
        while b >= a:
            #if check_solution(b) or check_solution(b + 1):
            if check_solution(b):
                #print(a, b, p)
                solutions.add((a, b))
                #solutions += 1
            p += 1
            b = infer(p, a)
        return len(solutions)
    a = 1
    solutions = 0
    while True:
        solutions += solve_for_a(a)
        a += 1
        p = int(10**n / a) + 1
        b = infer(p, a)
        if b < a:
            break
    print(f"{n}: {solutions}")
    return solutions

In [ ]:
[solve2(n) for n in range(1, 6)]

## For n = 1, why is 3, 6, 5 a solution, but 3, 7, p is not?

$10*(3 + 6) = p * 3 * 6 -> (2.5)(3.3) = (p)(3)(2.3)$

Cancelling factors:

$p = \frac{2.5.3.3}{3.2.3} = 5$

$10*(3 + 7) = p * 3 * 7 -> (2.5)(2.5) = (p)(3)(7)$

Cancelling factors:

$p = \frac{2.5.2.5}{3.7}$ -> does not reduce

# Check solutions for n = 2 

In [ ]:
def print_solutions(n):
    def f(n, a, pmin, bmax):
        for b in range(bmax, a - 1, -1):
            # if 10**n * (a + b) % (a * b) == 0:
            #     print(f"{10**n} x ({a} + {b}) = {10**n * (a + b)} = p x {a} x {b} --> p = {10**n * (a + b) / (a * b):.3f} --> {10**n * (a + b) % (a * b) == 0}")
            print(f"{10**n} x ({a} + {b}) = {10**n * (a + b)} = p x {a} x {b} --> p = {10**n * (a + b) / (a * b):.3f} --> {10**n * (a + b) % (a * b) == 0}")
    a = 1
    while True:
        pmin = ceil(10**n / a) if (10**n / a) < ceil(10**n / a) else ceil(10**n / a) + 1
        bmax = (10**n * a) // (pmin * a - 10**n)
        if a <= bmax:
            f(n, a, pmin, bmax)
            a += 1
        else:
            break

In [ ]:
def print_ranges(n):
    a = 1
    while True:
        pmin = ceil(10**n / a) if (10**n / a) < ceil(10**n / a) else ceil(10**n / a) + 1
        bmax = (10**n * a) // (pmin * a - 10**n)
        if a <= bmax:
            solutions = sum(1 for b in range(bmax, a - 1, -1) if 10**n * (a + b) % (a * b) == 0)
            print(f"n: {n}, a: {a}, minimum p : {pmin}, maximum b: {bmax}, number of solutions: {solutions}")
            a += 1
        else:
            break

In [ ]:
print_solutions(2)

In [ ]:
print_ranges(2)

### Taking a closer look at n = 2, a = 180

When $n=2$ and $a=180$, the minimum value for $p$ is $\lceil \frac {10^2}{180} \rceil=1$

The maximum value of $b$ is therefore $\frac{10^2  a}{p_{min} a - 10^2} = \frac{18,000}{180 - 100} = 225$

So we will end up evaluating all possibilities for $b$ in the range $[180, 225]$

In [ ]:
for b in range(180, 226):
    n, a = 2, 180
    print(f"{10**n} x ({a} + {b}) = {10**n * (a + b)} = p x {a} x {b} --> p = {10**n * (a + b) / (a * b):.3f} --> {10**n * (a + b) % (a * b) == 0}")

The values for $p$ are all non-integral except for $p = 1$, can we calculate a value $p_{max}$?

Rearrange equation as follows:

$10^n(a + b) = pab$

For a given value of $a$ we were calculating the minimum value of $p$ as follows:

$p_{min} = \frac{10^n}{a}$

But is this a minimum? This minimum is calculated based on the identity $b = 1$. But $b$ cannot be less than $a$.

So the minimum value of $pab$ is $pa^2$.

$10^n(a + a) = 2(10^n)a = pa^2 \Rightarrow p = \frac {2(10^n)}{a}$

In the example above ($n=2,a=180$), this would give $p_{min}= \lceil \frac{200}{180} \rceil = 1$

We know $p_{min}$ for given $a$; we can just take this value and keep incrementing it until we get an implied value for $b$ less than $a$.

In [ ]:
def print_ranges_alternate(n, show_solutions_only=False):
    a, s = 1, 0
    while True:
        p = 1
        while p * a - 10**n <= 0:
            p += 1
        if a > (10**n * a) / (p * a - 10**n):
            break
        while True:
            b = (10**n * a) / (p * a - 10**n)
            if b < a:
                break
            if b.is_integer():
                s += 1
            if show_solutions_only:
                if b.is_integer():
                    print(f"n: {n}, a: {a}, b: {b:.3f}, p : {p}, is solution: {b.is_integer()}")
            else:
                print(f"n: {n}, a: {a}, b: {b:.3f}, p : {p}, is solution: {b.is_integer()}")
            p += 1
        a += 1
    print('-' * 60)
    print(s)

In [ ]:
def print_solution_count_breakdown(n):
    a = 1
    while True:
        p, s = 1, 0
        while p * a - 10**n <= 0:
            p += 1
        if a > (10**n * a) / (p * a - 10**n):
            break
        while True:
            b = (10**n * a) / (p * a - 10**n)
            if b < a:
                break
            if b.is_integer():
                s += 1
            p += 1
        print(f"n: {n}, a: {a}, solution count: {s}")
        a += 1

In [ ]:
print_ranges_alternate(1)

In [ ]:
print_ranges_alternate(1, True)

In [ ]:
print_ranges_alternate(2, True)

In [ ]:
print_ranges_alternate(3, True)

In [ ]:
print_ranges_alternate(3)

# Conjecture

For some $n$ and $a$, if $a \cdot 10^n \mod (p \cdot a - 10^n) = 0$, then there is there is a solution $p, a, \frac{a.10^n}{(p.a - 10^n)}$

$p_{min} = \lceil \frac{10^n}{a} \rceil, p_{max} = 2 \cdot p_{min}$

(if $a \cdot p_{min} = 10^n$ then $p_{min} = p_{min} + 1$)

$a_{max} = 2 \cdot 10^n$

In [ ]:
for p in range(11, 21):
    n, a = 1, 1
    print(f"n: {n}, a: {a}, p: {p}, is solution: {(a * 10**n) % (p * a - 10**n) == 0}")

In [ ]:
for p in range(6, 11):
    n, a = 1, 2
    print(f"n: {n}, a: {a}, p: {p}, is solution: {(a * 10**n) % (p * a - 10**n) == 0}")

In [ ]:
for p in range(4, 7):
    n, a = 1, 3
    print(f"n: {n}, a: {a}, p: {p}, is solution: {(a * 10**n) % (p * a - 10**n) == 0}")

In [ ]:
def solve3(n):
    def solve_for_a(n, a):
        pmin = ceil(10**n / a)
        pmax = 2 * pmin
        if pmin * a == 10**n:
            pmin += 1
        s = 0
        for p in range(pmin, pmax + 1):
            if (a * 10**n) % (p * a - 10**n) == 0:
                s += 1
        return s
    return sum(solve_for_a(n, a) for a in range(1, 2 * 10**n + 1))